In [0]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import tensorflow as tf
from tensorflow import keras

# hyper parameters
embed_dim=100
oov_token='<OOv>'
trunc_type='post'
padding='post'
training_size=500000


In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/training_cleaned.csv \
    -O /tmp/training_cleaned.csv

--2020-06-01 09:14:06--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/training_cleaned.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 2607:f8b0:400e:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238942690 (228M) [application/octet-stream]
Saving to: ‘/tmp/training_cleaned.csv’

/tmp/training_clean 100%[===================>] 227.87M   140MB/s    in 1.6s    

2020-06-01 09:14:08 (140 MB/s) - ‘/tmp/training_cleaned.csv’ saved [238942690/238942690]



In [0]:
corpus=[]
with open('/tmp/training_cleaned.csv','r',encoding='utf-8') as csvdata:
  csvreader=csv.reader(csvdata,delimiter=',')
  for lines in csvreader:
    lists=[]
    sentence=lines[5]
    cat=int(lines[0])

    if cat == 0:
      labels=0
    else:
      labels=1
      
    lists.append(sentence)
    lists.append(labels)
    corpus.append(lists)

In [12]:
len(corpus)

1600000

In [0]:
import random
random.shuffle(corpus)
train_corpus=corpus[:training_size]

In [16]:
len(train_corpus)

500000

In [33]:
train_corpus[:2]

[['Ughhhhh waaaay too hot ', 0],
 ['@RockPaperAdam awe  are you having a shitty day too?', 0]]

In [0]:
test_size=0.1 * training_size
corpus_data,corpus_labels=[],[]
for data in train_corpus:
  corpus_data.append(data[0])
  corpus_labels.append(data[1])
  

In [0]:
from sklearn.model_selection import train_test_split
train_data,test_data,train_labels,test_labels=train_test_split(corpus_data,corpus_labels,test_size=0.1)

In [43]:
len(tain_data)

450000

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [0]:
token=Tokenizer(oov_token=oov_token)
token.fit_on_texts(corpus_data)
word_index=token.word_index

In [0]:
vocab_size=len(word_index)

In [48]:
vocab_size

312039

In [0]:
train_seq=token.texts_to_sequences(train_data)
test_seq=token.texts_to_sequences(test_data)

In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
train_pad_seq=pad_sequences(train_seq,maxlen=16,padding='post',truncating='post',)
test_pad_seq=pad_sequences(test_seq,maxlen=16,padding='post',truncating='post',)
train_labels=np.array(train_labels)
test_labels=np.array(test_labels)


In [55]:
train_pad_seq.shape,test_labels.shape

((450000, 16), (50000,))

In [56]:
### Lets load existing embedding weights
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt \
    -O /tmp/glove.6B.100d.txt

--2020-06-01 09:45:27--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347116733 (331M) [text/plain]
Saving to: ‘/tmp/glove.6B.100d.txt’

/tmp/glove.6B.100d. 100%[===================>] 331.04M   138MB/s    in 2.4s    

2020-06-01 09:45:30 (138 MB/s) - ‘/tmp/glove.6B.100d.txt’ saved [347116733/347116733]



In [0]:
embedding_matrix={}
with open('/tmp/glove.6B.100d.txt','r',encoding='utf-8') as glove:
  for lines in glove:
    data=lines.split()
    words=data[0]
    weights=np.array(data[1:],dtype=np.float32)
    embedding_matrix[words]=weights


In [0]:
embedding_vector=np.zeros((vocab_size+1,embed_dim),dtype=np.float32)
for word,i in word_index.items():
  word_exists = embedding_matrix.get(word)
  if word_exists is not None:
    embedding_vector[i] = embedding_matrix[word]


In [68]:
embedding_vector[5]

array([-0.27086  ,  0.044006 , -0.02026  , -0.17395  ,  0.6444   ,
        0.71213  ,  0.3551   ,  0.47138  , -0.29637  ,  0.54427  ,
       -0.72294  , -0.0047612,  0.040611 ,  0.043236 ,  0.29729  ,
        0.10725  ,  0.40156  , -0.53662  ,  0.033382 ,  0.067396 ,
        0.64556  , -0.085523 ,  0.14103  ,  0.094539 ,  0.74947  ,
       -0.194    , -0.68739  , -0.41741  , -0.22807  ,  0.12     ,
       -0.48999  ,  0.80945  ,  0.045138 , -0.11898  ,  0.20161  ,
        0.39276  , -0.20121  ,  0.31354  ,  0.75304  ,  0.25907  ,
       -0.11566  , -0.029319 ,  0.93499  , -0.36067  ,  0.5242   ,
        0.23706  ,  0.52715  ,  0.22869  , -0.51958  , -0.79349  ,
       -0.20368  , -0.50187  ,  0.18748  ,  0.94282  , -0.44834  ,
       -3.6792   ,  0.044183 , -0.26751  ,  2.1997   ,  0.241    ,
       -0.033425 ,  0.69553  , -0.64472  , -0.0072277,  0.89575  ,
        0.20015  ,  0.46493  ,  0.61933  , -0.1066   ,  0.08691  ,
       -0.4623   ,  0.18262  , -0.15849  ,  0.020791 ,  0.1937

In [0]:
from tensorflow.keras.layers import Dense,Embedding,LSTM,Conv1D,AveragePooling1D

In [72]:
model=keras.Sequential()
model.add(Embedding(input_dim=vocab_size + 1,output_dim=embed_dim,input_length=16,weights=[embedding_vector],trainable=False))
model.add(Conv1D(filters=64,kernel_size=5,activation=tf.nn.relu))
model.add(AveragePooling1D(pool_size=4))
model.add(LSTM(64))
model.add(Dense(units=1,activation=tf.nn.sigmoid))
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 16, 100)           31204000  
_________________________________________________________________
conv1d (Conv1D)              (None, 12, 64)            32064     
_________________________________________________________________
average_pooling1d (AveragePo (None, 3, 64)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                33024     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 31,269,153
Trainable params: 65,153
Non-trainable params: 31,204,000
_________________________________________________________________


In [74]:
model.fit(train_pad_seq,train_labels,epochs=30,validation_data=(test_pad_seq,test_labels))

Epoch 1/30
14063/14063 [==============================] - 56s 4ms/step - loss: 0.6320 - accuracy: 0.6992 - val_loss: 0.6260 - val_accuracy: 0.7365
Epoch 2/30
14063/14063 [==============================] - 56s 4ms/step - loss: 0.6197 - accuracy: 0.7308 - val_loss: 0.6201 - val_accuracy: 0.7378
Epoch 3/30
14063/14063 [==============================] - 56s 4ms/step - loss: 0.6150 - accuracy: 0.7422 - val_loss: 0.6216 - val_accuracy: 0.7495
Epoch 4/30
14063/14063 [==============================] - 56s 4ms/step - loss: 0.6115 - accuracy: 0.7509 - val_loss: 0.6172 - val_accuracy: 0.7299
Epoch 5/30
14063/14063 [==============================] - 56s 4ms/step - loss: 0.6089 - accuracy: 0.7579 - val_loss: 0.6175 - val_accuracy: 0.7281
Epoch 6/30
14063/14063 [==============================] - 56s 4ms/step - loss: 0.6068 - accuracy: 0.7631 - val_loss: 0.6156 - val_accuracy: 0.7491
Epoch 7/30
14063/14063 [==============================] - 56s 4ms/step - loss: 0.6052 - accuracy: 0.7668 - val_loss: 0

In [76]:
model1=keras.Sequential()
model1.add(Embedding(input_dim=vocab_size + 1,output_dim=embed_dim,input_length=16,weights=[embedding_vector],trainable=False))
model1.add(Conv1D(filters=64,kernel_size=5,activation=tf.nn.relu))
model1.add(AveragePooling1D(pool_size=2))
model1.add(Conv1D(filters=32,kernel_size=5,activation=tf.nn.relu))
model1.add(AveragePooling1D(pool_size=2))
model1.add(LSTM(32))
model1.add(Dense(units=1,activation=tf.nn.sigmoid))
model1.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
model1.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 16, 100)           31204000  
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 12, 64)            32064     
_________________________________________________________________
average_pooling1d_3 (Average (None, 6, 64)             0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 2, 32)             10272     
_________________________________________________________________
average_pooling1d_4 (Average (None, 1, 32)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                

In [77]:
model.fit(train_pad_seq,train_labels,epochs=30,validation_data=(test_pad_seq,test_labels))

Epoch 1/30
14063/14063 [==============================] - 63s 4ms/step - loss: 0.6328 - accuracy: 0.6975 - val_loss: 0.6300 - val_accuracy: 0.6868
Epoch 2/30
14063/14063 [==============================] - 62s 4ms/step - loss: 0.6207 - accuracy: 0.7278 - val_loss: 0.6239 - val_accuracy: 0.7031
Epoch 3/30
14063/14063 [==============================] - 62s 4ms/step - loss: 0.6161 - accuracy: 0.7396 - val_loss: 0.6271 - val_accuracy: 0.7479
Epoch 4/30
14063/14063 [==============================] - 62s 4ms/step - loss: 0.6127 - accuracy: 0.7479 - val_loss: 0.6171 - val_accuracy: 0.7335
Epoch 5/30
14063/14063 [==============================] - 61s 4ms/step - loss: 0.6103 - accuracy: 0.7545 - val_loss: 0.6165 - val_accuracy: 0.7377
Epoch 6/30
14063/14063 [==============================] - 63s 4ms/step - loss: 0.6081 - accuracy: 0.7594 - val_loss: 0.6169 - val_accuracy: 0.7510
Epoch 7/30
14063/14063 [==============================] - 61s 4ms/step - loss: 0.6065 - accuracy: 0.7628 - val_loss: 0

In [79]:
model2=keras.Sequential()
model2.add(Embedding(input_dim=vocab_size + 1,output_dim=embed_dim,input_length=16,weights=[embedding_vector],trainable=False))
model2.add(Conv1D(filters=64,kernel_size=5,activation=tf.nn.relu))
model2.add(AveragePooling1D(pool_size=2))
model2.add(Conv1D(filters=32,kernel_size=5,activation=tf.nn.relu))
model2.add(AveragePooling1D(pool_size=2))
model2.add(LSTM(32))
model2.add(Dense(units=12,activation=tf.nn.sigmoid))
model2.add(Dense(units=1,activation=tf.nn.sigmoid))
model2.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
model2.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 16, 100)           31204000  
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 12, 64)            32064     
_________________________________________________________________
average_pooling1d_7 (Average (None, 6, 64)             0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 2, 32)             10272     
_________________________________________________________________
average_pooling1d_8 (Average (None, 1, 32)             0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_4 (Dense)              (None, 12)               

In [80]:
model.fit(train_pad_seq,train_labels,epochs=30,validation_data=(test_pad_seq,test_labels))

Epoch 1/30
14063/14063 [==============================] - 62s 4ms/step - loss: 0.5913 - accuracy: 0.7993 - val_loss: 0.6167 - val_accuracy: 0.7386
Epoch 2/30
14063/14063 [==============================] - 62s 4ms/step - loss: 0.5910 - accuracy: 0.8004 - val_loss: 0.6152 - val_accuracy: 0.7478
Epoch 3/30
14063/14063 [==============================] - 62s 4ms/step - loss: 0.5909 - accuracy: 0.7999 - val_loss: 0.6157 - val_accuracy: 0.7518
Epoch 4/30
14063/14063 [==============================] - 62s 4ms/step - loss: 0.5907 - accuracy: 0.8005 - val_loss: 0.6158 - val_accuracy: 0.7526
Epoch 5/30
14063/14063 [==============================] - 63s 4ms/step - loss: 0.5900 - accuracy: 0.8021 - val_loss: 0.6161 - val_accuracy: 0.7428
Epoch 6/30
14063/14063 [==============================] - 62s 4ms/step - loss: 0.5899 - accuracy: 0.8024 - val_loss: 0.6161 - val_accuracy: 0.7478
Epoch 7/30
14063/14063 [==============================] - 62s 4ms/step - loss: 0.5898 - accuracy: 0.8021 - val_loss: 0